## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

## Import Data File

In [2]:
df_housing = pd.read_csv('C:/Users/Ayman/Desktop/Data Analysis/Courses/13+ Power BI Portfolio Projects with DAX & SQL (2025)/Project-3_Housing Data Analysis (Data Source Google Big Query)/Housing Data.csv')
df_housing.head()

,date,quarter,house_id,house_type,sales_type,year_build,purchase_price,%_change_between_offer_and_purchase,no_rooms,sqm,sqm_price,address,zip_code,city,area,region,nom_interest_rate%,dk_ann_infl_rate%,yield_on_mortgage_credit_bonds%
0,10/24/2024,2024Q4,16,Villa,regular_sale,1997,6500000,-3,5,142,45774.650,Spættevej 16,2680,Solrød Strand,"Capital, Copenhagen",Zealand,3.1,NaN,NaN
1,10/24/2024,2024Q4,13,Apartment,regular_sale,1885,3400000,0,2,46,73913.050,"Jakob Dannefærds Vej 10B, 4. th",1973,Frederiksberg C,"Capital, Copenhagen",Zealand,3.1,NaN,NaN
2,10/23/2024,2024Q4,60,Villa,regular_sale,1949,4550000,-4,4,112,40625.000,Vestre Plantagevej 3,2680,Solrød Strand,"Capital, Copenhagen",Zealand,3.1,NaN,NaN
3,10/23/2024,2024Q4,29,Villa,regular_sale,2001,1630000,-12,4,186,8763.440,Toftebjerg 3,4400,Kalundborg,Other islands,Zealand,3.1,NaN,NaN
4,10/22/2024,2024Q4,92,Apartment,regular_sale,1965,1975000,-10,3,82,24085.365,"Ryttergårdsvej 20, 1. 113",3520,Farum,North Zealand,Zealand,3.1,NaN,NaN


## Data Cleaning & Transformation

In [3]:
# Check the basic statistics
print("\nDescriptive Statistics:")
print(df_housing.describe(include='all'))


Descriptive Statistics:
              date quarter      house_id house_type    sales_type  \
count       100000  100000  1.000000e+05     100000        100000   
unique       11254     130           NaN          5             4   
top     12/19/2023  2021Q1           NaN      Villa  regular_sale   
freq            49    2263           NaN      54214         88232   
mean           NaN     NaN  7.531364e+05        NaN           NaN   
std            NaN     NaN  4.348902e+05        NaN           NaN   
min            NaN     NaN  1.300000e+01        NaN           NaN   
25%            NaN     NaN  3.770722e+05        NaN           NaN   
50%            NaN     NaN  7.511035e+05        NaN           NaN   
75%            NaN     NaN  1.128904e+06        NaN           NaN   
max            NaN     NaN  1.507901e+06        NaN           NaN   

           year_build  purchase_price  %_change_between_offer_and_purchase  \
count   100000.000000    1.000000e+05                        100000.

In [4]:
# Check for missing values
print("\nMissing Values:")
missing_data = df_housing.isnull().sum()
missing_percent = (df_housing.isnull().sum() / len(df_housing)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])


Missing Values:
                                 Missing Count  Missing Percentage
city                                        11               0.011
dk_ann_infl_rate%                           77               0.077
yield_on_mortgage_credit_bonds%             77               0.077


In [5]:
# Get rows where city is null, showing zip_code and other relevant columns
null_cities = df_housing[df_housing['city'].isnull()][['city', 'zip_code', 'address', 'region', 'area']]

print("Rows with missing city values:")
print(f"Total missing cities: {len(null_cities)}")
print("\nDetailed view:")
print(null_cities)

Rows with missing city values:
Total missing cities: 11

Detailed view:
      city  zip_code             address   region                area
7358   NaN      8799         Søndenom 11  Jutland  East & mid jutland
12234  NaN      8799   Tunø Hovedgade 37  Jutland  East & mid jutland
16138  NaN      8799            Revet 20  Jutland  East & mid jutland
35666  NaN      8799          Avrevej 75  Jutland  East & mid jutland
44934  NaN      8799            Revet 35  Jutland  East & mid jutland
53125  NaN      8799          Søndenom 5  Jutland  East & mid jutland
63952  NaN      8799   Tunø Hovedgade 16  Jutland  East & mid jutland
65377  NaN      8799   Tunø Hovedgade 72  Jutland  East & mid jutland
66061  NaN      8799       Stenkalven 71  Jutland  East & mid jutland
69069  NaN      8799  Tunø Hovedgade 100  Jutland  East & mid jutland
81161  NaN      8799            Revet 18  Jutland  East & mid jutland


In [6]:
# Get unique zip codes that have missing cities
missing_zip_codes = null_cities['zip_code'].unique()
print(f"Unique zip codes with missing cities: {missing_zip_codes}")

# Check if these zip codes exist in the complete data with known cities
for zip_code in missing_zip_codes:
    zip_code_data = df_housing[df_housing['zip_code'] == zip_code]
    known_cities = zip_code_data[zip_code_data['city'].notnull()]['city'].unique()
    
    print(f"\nZip Code: {zip_code}")
    print(f"Total records with this zip: {len(zip_code_data)}")
    print(f"Records with missing city: {len(zip_code_data[zip_code_data['city'].isnull()])}")
    print(f"Known cities for this zip: {known_cities}")
    
    # If there's a consistent city for this zip code, we can use it for imputation
    if len(known_cities) == 1:
        print(f"Consistent city found: {known_cities[0]} - Can be used for imputation")
    elif len(known_cities) > 1:
        print(f"Multiple cities for this zip: {known_cities} - Need investigation")
    else:
        print(f"No known cities for this zip code in dataset")

Unique zip codes with missing cities: [8799]

Zip Code: 8799
Total records with this zip: 11
Records with missing city: 11
Known cities for this zip: []
No known cities for this zip code in dataset


In [7]:
# Replacing the missing city values
df_housing.loc[df_housing['city'].isnull(), 'city'] = 'Tunø'

# After replacement - verifying the changes
print("\nAfter replacement:")
print(f"Missing cities: {df_housing['city'].isnull().sum()}")
print(f"City value counts for 'Tunø': {df_housing[df_housing['city'] == 'Tunø'].shape[0]}")


After replacement:
Missing cities: 0
City value counts for 'Tunø': 11


In [8]:
# Get rows with missing inflation rates
missing_inflation = df_housing[df_housing['dk_ann_infl_rate%'].isnull()]

print(f"Total missing inflation rates: {len(missing_inflation)}")
print(f"Date range with missing inflation: {missing_inflation['date'].min()} to {missing_inflation['date'].max()}")

Total missing inflation rates: 77
Date range with missing inflation: 10/1/2024 to 10/9/2024


In [9]:
# Extract year from date for better analysis
df_housing['year'] = pd.to_datetime(df_housing['date']).dt.year

# Check 2024 data specifically
data_2024 = df_housing[df_housing['year'] == 2024]

print(f"Total records from 2024: {len(data_2024)}")
print(f"Records with inflation data: {data_2024['dk_ann_infl_rate%'].notnull().sum()}")
print(f"Records missing inflation data: {data_2024['dk_ann_infl_rate%'].isnull().sum()}")

# Check if there are any non-null inflation values for 2024 and replace them
existing_2024_inflation = data_2024['dk_ann_infl_rate%'].dropna()

if len(existing_2024_inflation) > 0:
    print(f"Existing inflation rate(s) for 2024 in dataset: {existing_2024_inflation.unique()}")
    inflation_2024 = existing_2024_inflation.iloc[0]
    
    print(f"\nReplacing missing 2024 inflation values with: {inflation_2024}%")
    
    # Before replacement
    print(f"Missing 'dk_ann_infl_rate%' before: {df_housing['dk_ann_infl_rate%'].isnull().sum()}")
    
    # Replacing the missing values
    df_housing.loc[(df_housing['dk_ann_infl_rate%'].isnull()) & (df_housing['year'] == 2024), 'dk_ann_infl_rate%'] = inflation_2024
    
    # After replacement
    print(f"Missing 'dk_ann_infl_rate%' after: {df_housing['dk_ann_infl_rate%'].isnull().sum()}")
    
else:
    print("\nNo existing inflation data for 2024 in the dataset")

Total records from 2024: 4108
Records with inflation data: 4031
Records missing inflation data: 77
Existing inflation rate(s) for 2024 in dataset: [1.13]

Replacing missing 2024 inflation values with: 1.13%
Missing 'dk_ann_infl_rate%' before: 77
Missing 'dk_ann_infl_rate%' after: 0


In [10]:
# Get rows with missing mortgage rates
missing_mortgage = df_housing[df_housing['yield_on_mortgage_credit_bonds%'].isnull()]

print(f"Total missing mortgage rates: {len(missing_mortgage)}")
print(f"Date range with missing mortgage: {missing_mortgage['date'].min()} to {missing_mortgage['date'].max()}")

Total missing mortgage rates: 77
Date range with missing mortgage: 10/1/2024 to 10/9/2024


In [11]:
print(f"Total records from 2024: {len(data_2024)}")
print(f"Records with mortgage data: {data_2024['yield_on_mortgage_credit_bonds%'].notnull().sum()}")
print(f"Records missing mortgage data: {data_2024['yield_on_mortgage_credit_bonds%'].isnull().sum()}")

# Check if there are any non-null mortgage values for 2024 and replace them
existing_2024_mortgage = data_2024['yield_on_mortgage_credit_bonds%'].dropna()

if len(existing_2024_mortgage) > 0:
    print(f"Existing mortgage rate(s) for 2024 in dataset: {existing_2024_mortgage.unique()}")
    mortgage_2024 = existing_2024_mortgage.iloc[0]
    
    print(f"\nReplacing missing 2024 mortgage values with: {mortgage_2024}%")
    
    # Before replacement
    print(f"Missing 'yield_on_mortgage_credit_bonds%' before: {df_housing['yield_on_mortgage_credit_bonds%'].isnull().sum()}")
    
    # Replacing the missing values
    df_housing.loc[(df_housing['yield_on_mortgage_credit_bonds%'].isnull()) & (df_housing['year'] == 2024), 'yield_on_mortgage_credit_bonds%'] = mortgage_2024
    
    # After replacement
    print(f"Missing 'yield_on_mortgage_credit_bonds%' after: {df_housing['yield_on_mortgage_credit_bonds%'].isnull().sum()}")
    
else:
    print("\nNo existing mortgage data for 2024 in the dataset")

Total records from 2024: 4108
Records with mortgage data: 4031
Records missing mortgage data: 77
Existing mortgage rate(s) for 2024 in dataset: [4.34]

Replacing missing 2024 mortgage values with: 4.34%
Missing 'yield_on_mortgage_credit_bonds%' before: 77
Missing 'yield_on_mortgage_credit_bonds%' after: 0


In [12]:
# Check for duplicated values in the 'house_id' column
house_id_duplicates = df_housing.duplicated(subset=['house_id']).sum()
print(f"Duplicate house_id values: {house_id_duplicates}")

if house_id_duplicates > 0:
    print("\nRows with duplicate house_id:")
    duplicate_house_ids = df[df.duplicated(subset=['house_id'], keep=False)]
    print(duplicate_house_ids.sort_values('house_id').head(20))
else:
    print("No duplicate house_id values found.")

Duplicate house_id values: 0
No duplicate house_id values found.


In [13]:
# Save the DataFrame as CSV
df_housing.to_csv('housing_cleaned.csv', index=False)
print(f"Number of rows saved: {len(df_housing)}")
print(f"Number of columns saved: {len(df_housing.columns)}")

Number of rows saved: 100000
Number of columns saved: 20
